# Food Scanning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import PIL
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

## Preprocessing Image

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "../input/food-ingredients/Data/train"
VALIDATION_DIR = "../input/food-ingredients/Data/val"

# Make Labels
classes = os.listdir(TRAINING_DIR)
with open('food_label.txt', 'w') as f:
    for food_class in classes:
        f.write(f'{food_class}\n')

train_ds = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      )

train_generator = train_ds.flow_from_directory(directory=TRAINING_DIR,
                                                batch_size=128,
                                                class_mode='categorical',
                                                target_size=(256, 256))

test_ds = ImageDataGenerator(
      rescale = 1./255)
validation_generator = test_ds.flow_from_directory(directory=VALIDATION_DIR,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    target_size=(256, 256))

## Build Model

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    restore_best_weights=True
)

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.7),
    # The fifth convolution
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The sixth convolution
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    # 320 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(26, activation='softmax')
])

# Print the model summary
model.summary()

In [ ]:
# Set the training parameters

#opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training Model

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=75, validation_data = validation_generator, callbacks = [callbacks])

## Plotting Training and Validation

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()

## Save and Convert 

In [ ]:
#Save the model
#!mkdir -p saved_model
model.save('saved_model/val_acc8447')

# Convert the model
path = '/kaggle/working/saved_model/val_acc8447'
converter = tf.lite.TFLiteConverter.from_saved_model(path) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('food_val8447.tflite', 'wb') as f:
  f.write(tflite_model)
